# Recursive Water and Land Mapping

## Import required packages and functions

For details on the installation procedure, please check the `README.md` file of tis project.

In [ ]:
import random
import os

from configuration import Config, Debug
from image_reader import ReadSentinel2
from training import training_main
from evaluation import evaluation_main
from datetime import datetime

## Download Dataset

Download our dataset *Sentinel-2 Images from Oroville Dam and Charles River* from [this Zenodo link](https://zenodo.org/record/6999172#.YzWyndjMI2x) and extract the `.zip` file. 
In `configuration.py` (class `Config`), change `path_sentinel_images` to the `sentinel_data` folder path. Images in this dataset are used for training and evaluation. Details regarding the dataset can be found in the Zenodo link.

## Configuration

Please, consider changing the configuration settings from the `configuration.py` script as desired. Some configuration settings must be changed when executing the code with data that is different to the one provided by the authors.

For simplicity, some configuration settings are overwritten in this file. 

### Paths
Please, overwrite the `Config.path_sentinel_images` parameter with the corresponding path where the dataset images downloaded from the Zenodo link have been stored.


In [ ]:
Config.path_sentinel_images = r"C:\Users\HNereida\Documents\Northeastern\hd-img\Sentinel2_data"  # Change
Config.path_watnet_pretrained_model = os.path.join(os.getcwd(), r"benchmark\watnet\model\pretrained\watnet.h5")
Config.path_log_files = os.path.join(os.getcwd(), r"logs")
Config.path_trained_models = os.path.join(os.getcwd(), r"trained_models")

### Debugging Options 

Set the following debugging options to `True` if wanting to train the GMM and LR models. The trained models will be stored in pickle files within the `Config.path_trained_models` folder. 

If the models have already been trained and stored in pickle files within that folder, the following debugging options can be set to `False` in order to skip the training stage. Authors provide already trained models so that the results provided in the publication can be reproduced.

In [ ]:
Debug.gmm_dump_pickle = False
Debug.trained_lr_model_pickle = False

## Main Function
This function contains the main logic and flow of the code.

In [ ]:
# Initialize random seed
random.seed(1)

# Set logging path
Debug.set_logging_file(time_now=datetime.now())

# Instance of Image Reader object
image_reader = ReadSentinel2(Config.image_dimensions[Config.scenario]['dim_x'],
                             Config.image_dimensions[Config.scenario]['dim_y'])

### Training Stage

In [ ]:
# Training Stage
labels, gmm_densities, trained_lr_model = training_main(image_reader)

### Evaluation Stage

In [ ]:
# Evaluation Stage
# This stage includes the plotting of results
evaluation_main(gmm_densities, trained_lr_model, image_reader)


## Authors
* Bhavya Duvvuri, from The Beighley Lab (Sustainable Water Resources | Resilient Wet Infrastructure) at Northeastern
University, Boston (MA).
* Helena Calatrava, from the Signal Processing, Imaging, Reasoning and Learning (SPIRAL) Group also at 
Northeastern University, Boston (MA). 
